#### Azure Batch Shipyard devito 

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
DOTENV_FILE_PATH = ['.', 'not_shared'] + ['general.env']

docker_file_location = 'docker_build'
docker_build_no_cache = ''  # '--no-cache' # or '' #

In [3]:
# dockerhub_login = "georgedockeraccount"
# dockerhub_pwd = ""

# ACR_NAME="fwi01acr"
# ACR_PASSWORD=""
# ACR_USERNAME="fwi01acr"

# control_plane_docker_image_name = "azure_batch_shipyard_devito"
# control_plane_docker_image_tag = ""

In [4]:
import sys, os, pathlib
import platform,  dotenv
import subprocess

In [5]:
platform.platform()
os.getcwd()

'Linux-4.15.0-1066-azure-x86_64-with-debian-stretch-sid'

'/datadrive01/prj/Azure2019/notebooks'

In [6]:
dotenv_file_path = os.path.join(*(DOTENV_FILE_PATH)) 
os.makedirs(os.path.join(*(DOTENV_FILE_PATH[:-1])), exist_ok=True)
pathlib.Path(dotenv_file_path).touch()

# # show .env file path
# !pwd
dotenv_file_path


'./not_shared/general.env'

Save your sensitive info

In [7]:
# dotenv.set_key(dotenv_file_path, 'DOCKERHUB_LOGIN', dockerhub_login)
# dotenv.set_key(dotenv_file_path, 'DOCKERHUB_PWD', dockerhub_pwd)

# dotenv.set_key(dotenv_file_path, 'ACR_NAME', ACR_NAME)
# dotenv.set_key(dotenv_file_path, 'ACR_PASSWORD', ACR_PASSWORD)
# dotenv.set_key(dotenv_file_path, 'ACR_USERNAME', ACR_USERNAME)

# dotenv.set_key(dotenv_file_path,'control_plane_docker_image_name', control_plane_docker_image_name)
# dotenv.set_key(dotenv_file_path,'control_plane_docker_image_tag', control_plane_docker_image_tag)

In [8]:
dotenv.load_dotenv(dotenv_path=dotenv_file_path)
docker_file_location = os.path.join(*(os.getcwd(), docker_file_location))

docker_file_name = 'Dockerfile'+ '_' + os.getenv('control_plane_docker_image_name')

docker_repo_name = os.getenv('DOCKERHUB_LOGIN') # or  os.getenv('ACR_NAME')+'.azurecr.io'
docker_image_name = docker_repo_name + '/' + os.getenv('control_plane_docker_image_name')

image_version = os.getenv('control_plane_docker_image_tag')
if image_version!="":
    docker_file_name = docker_file_name +'_'+ image_version
    docker_image_name = docker_image_name +':'+ image_version

docker_file_dir = os.path.join(*([os.getcwd(), docker_file_location]))
os.makedirs(docker_file_dir, exist_ok=True)
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))

docker_image_name
docker_file_dir
docker_file_path

True

'georgedockeraccount/azure_batch_shipyard_devito'

'/datadrive01/prj/Azure2019/notebooks/docker_build'

'/datadrive01/prj/Azure2019/notebooks/docker_build/Dockerfile_azure_batch_shipyard_devito'

Dockerfile references:  
https://github.com/Azure/batch-shipyard/blob/master/images/docker/linux/cli/Dockerfile  
https://github.com/matsuu/docker-azure-cli/blob/master/Dockerfile  
https://github.com/Azure/azure-cli/blob/dev/Dockerfile  
https://github.com/Azure/azure-functions-core-tools#other-linux-distributions  

In [9]:
%%writefile $docker_file_path 

# https://hub.docker.com/r/microsoft/azure-cli/dockerfile   
FROM debian:latest 
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

# https://github.com/Azure/batch-shipyard/blob/master/images/docker/linux/cli/Dockerfile
# https://github.com/Azure/batch-shipyard/blob/master/cargo/Dockerfile
RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    python3 python3-pip python3-setuptools \
    ca-certificates curl apt-transport-https lsb-release gnupg \
    zip unzip \
    wget rsync git && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*   && \
    git clone https://github.com/Azure/batch-shipyard.git /opt/batch-shipyard && \
    cd /opt/batch-shipyard && \
    rm -rf .git .github .vsts && \
    rm -f .git* .travis.yml *.yml install*  && \
    python3 -m pip install --no-cache-dir --upgrade pip  && \
    pip3 install --no-cache-dir -r requirements.txt  && \
    pip3 install --no-cache-dir --no-deps -r req_nodeps.txt && \
    python3 -m compileall -f /opt/batch-shipyard/shipyard.py /opt/batch-shipyard/convoy 
# ENTRYPOINT ["/opt/batch-shipyard/shipyard.py"]

# https://docs.microsoft.com/en-us/cli/azure/install-azure-cli-apt?view=azure-cli-latest#no-package-for-your-distribution
# ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $(lsb_release -sc) main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli 

# https://github.com/Azure/azure-functions-core-tools
ENV AzFunctionsCoreTools_DIR /azure-functions-cli
RUN mkdir /tmp/AzFunctionsCoreTools3 && \
    mkdir -p $AzFunctionsCoreTools_DIR && \
    cd /tmp/AzFunctionsCoreTools3 && \
    wget --quiet --no-check-certificate \
    https://github.com/Azure/azure-functions-core-tools/releases/download/2.7.2100/Azure.Functions.Cli.linux-x64.2.7.2100.zip  && \
    unzip  -qq -d $AzFunctionsCoreTools_DIR Azure.Functions.Cli.linux-x64.*.zip && \
    rm -f Azure.Functions.Cli.linux-x64.*.zip && \
    cd $AzFunctionsCoreTools_DIR && \
    chmod +x func
ENV PATH=$AzFunctionsCoreTools_DIR:$PATH    

Overwriting /datadrive01/prj/Azure2019/notebooks/docker_build/Dockerfile_azure_batch_shipyard_devito


In [10]:
cli_command='docker build -t '+ docker_image_name + \
' -f ' + docker_file_path + \
' ' + docker_file_dir + ' ' +\
docker_build_no_cache  + ' ' 
#  ' --target AzureBatchShipyardAzCliAzFunctionsCoreTools'

cli_command
# docker_build_response = ! $cli_command

# docker_build_response[0:5] 
# docker_build_response[-5:]

'docker build -t georgedockeraccount/azure_batch_shipyard_devito -f /datadrive01/prj/Azure2019/notebooks/docker_build/Dockerfile_azure_batch_shipyard_devito /datadrive01/prj/Azure2019/notebooks/docker_build  '

In [11]:
! $cli_command

Sending build context to Docker daemon  4.096kB
Step 1/7 : FROM debian:latest
 ---> b5d2d9b1597b
Step 2/7 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> e7d6a00c9a36
Step 3/7 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     python3 python3-pip python3-setuptools     ca-certificates curl apt-transport-https lsb-release gnupg     zip unzip     wget rsync git &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*   &&     git clone https://github.com/Azure/batch-shipyard.git /opt/batch-shipyard &&     cd /opt/batch-shipyard &&     rm -rf .git .github .vsts &&     rm -f .git* .travis.yml *.yml install*  &&     python3 -m pip install --no-cache-dir --upgrade pip  &&     pip3 install --no-cache-dir -r requirements.txt  &&     pip3 install --no-cache-dir --no-deps -r req_nodeps.txt &&     python3 -m compileall -f /opt/batch-shipyard/shipyard.py /opt/batch-shipyard/convoy
 ---> Using cache
 ---> 0467118246d1
Step 4/7 : RUN e

In [ ]:
# docker run \
# -it \
# --rm  \
# --name azure-cli_container01 \
# -v $(pwd):/workspace:rw \
# -v /usr/bin/docker:/usr/bin/docker \
# -v /var/run/docker.sock:/var/run/docker.sock \
# georgedockeraccount/azure_batch_shipyard_devito \
# /bin/bash -c "cd /workspace/src/AzureBatch/docker/tti_image/ &&  docker build -t devito_azure_tti:v1.0 ."
